# 🔍 GenAI-Powered Credit Decisioning in Modern Banking

The financial services industry is undergoing a fundamental transformation as banks leverage Generative AI to revolutionize credit decisioning processes. Traditional systems based on static rules and manual reviews are being replaced by intelligent automation that delivers unprecedented speed, accuracy, and transparency. Modern banks using AI underwriting report 50-75% reductions in time-to-decision while improving risk assessment and expanding access to credit for underserved populations.

<div style="background: #f7f7f7; border-left: 5px solid #ff5f46; padding: 20px; margin: 20px 0; font-size: 18px;"> "By using machine learning models, financial institutions can accurately predict the likelihood of a borrower defaulting on a loan. This predictive capability is grounded in real data, minimizing guesswork and speculation...AI algorithms process credit applications much quicker than human analysts, slashing decision times and enhancing customer satisfaction." <br> <span style="font-size:16px;">— nCino, The Revolution of AI Credit Decisioning in Banking (2024)</span> </div> <div style="background: #f7f7f7; border-left: 5px solid #21b86b; padding: 20px; margin: 20px 0; font-size: 18px;"> "With the Databricks Data Intelligence Platform for Financial Services, Discovery Bank accelerates data analytics and AI to drive business value, streamline governance and automate complex tasks. Our joint efforts demonstrate how data can drive transformation and add value to society." <br> <span style="font-size:16px;">— Dael Williamson, Field CTO, Databricks (2024)</span> </div>

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="width: 100px; vertical-align: middle; margin-right: 10px; float: left;" />
<br>
Liza, a Data Engineer, is using Databricks and generative AI to modernize credit decisioning. Her workflow:

<div style="background: #e0f7fa; border-left: 5px solid #00796b; padding-left: 10px; margin: 0 0; margin-left: 120px">
  <ol style="text-align: left;">
    <li>Fetch customer credit information with AI Functions</li>
    <li>Generate structured credit decision reports using generative AI</li>
    <li>Log all AI functions and outputs in Unity Catalog for compliance</li>
    <li>Apply these AI functions in batch to analyze all potentially fraudulent transactions</li>
  </ol>
</div>



## The Demo: Automate Credit Decision Reports with Databricks GenAI


In this demo, we showcase a practical use case for Databricks to improve credit decision system in modern banking. We will create an AI agent using the following functions, defined in the subsequent cells:

1. **Customer Credit Information Retriever**: This function allows our GenAI agent to fetch detailed customer credit information for any given customer id, pulling structured and unstructured data from our Databricks Intelligence Platform.

2. **Credit Decision Report Generator**: Using generative AI models, this function creates clear, structured credit decision reports , highlighting risk factors, behavioral anomalies, and relevant customer context for further review.

Finally, we will demonstrate how these functions can be applied in batch to analyze all the credit decisions for a customer. By automating credit decision report generation and enabling real-time access to enriched data, banks can improve operational efficiency, accelerate decision making power, and enhance compliance.

By leveraging Generative AI, banks can move beyond manual processes to **streamline workflows, reduce costs, and significantly strengthen Credit Risk Analysis capabilities**.

In [0]:
%run ../_resources/00-setup $reset_all_data=false

## 📊 Function 1: Get Customer Details

<div style="display: flex; align-items: center; gap: 15px;"> <img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="width: 100px; border-radius: 10px;" /> <div> Liza's first step in automating credit risk analysis is to retrieve all relevant information for a given customer.<br> This function, <code>get_customer_details</code>, takes in a customer ID and returns the necessary data to generate a comprehensive credit decision report.<br> The retrieved data includes customer profile information, financials and transaction history for accurate and efficient credit risk detection and analysis. </div> </div>

### 🛠️ Next Step
In the following section, we define the <code>get_customer_details</code> function, which retrieves structured data from the Databricks Lakehouse to enable automated credit risk detection and report generation.

In [0]:
%sql
DROP FUNCTION IF EXISTS get_customer_details;

CREATE OR REPLACE FUNCTION get_customer_details (
  customer_id STRING COMMENT 'Customer ID of the customer to be searched'
) 
RETURNS TABLE(
  cust_id INT,
  education INT,
  marital_status INT,
  age INT,
  is_resident INT,
  months_current_address INT,
  months_employment INT,
  number_payment_delays_last12mo BIGINT,
  pct_increase_annual_number_of_delays_last_3_year BIGINT,
  overdraft_balance_amount DOUBLE,
  tot_rel_bal DOUBLE,
  tot_assets DOUBLE,
  income_annual INT,
  avg_balance DOUBLE,
  num_accs BIGINT,
  tot_txn_amt_12m DOUBLE,
  sent_txn_cnt_12m BIGINT,
  prediction DOUBLE

)
COMMENT "This function returns the demographic and financial information about customers for a specific customer ID" 
RETURN (
  SELECT
    cust_id,
    education,
    marital_status,
    age,
    is_resident,
    months_current_address,
    months_employment,
    number_payment_delays_last12mo,
    pct_increase_annual_number_of_delays_last_3_year,
    overdraft_balance_amount,
    tot_rel_bal,
    tot_assets,
    income_annual,
    avg_balance,
    num_accs,
    tot_txn_amt_12m,
    sent_txn_cnt_12m,
    prediction

  FROM 
    underbanked_prediction
  WHERE
    cust_id = customer_id
)

In [0]:
%sql
SELECT * FROM get_customer_details(
    '93407'
) AS prediction


## 📝 Function 2: Generate Credit Decision Report
<div style="display: flex; align-items: center; gap: 15px;"> <img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="width: 100px; border-radius: 10px;" /> <div> After gathering customer and their financial transaction details, Liza’s next step is to automate the creation of professional reports with Credit Decision & Explanation.<br> The <code>credit_report_generator</code> function leverages generative AI to analyze transaction risk and produce clear, actionable documentation for both internal teams and customers.<br> This function: <ul> <li>Predicts whether a customer's credit risk is low or high using advanced AI models.</li> <li>Automatically generates a formal internal  report for the bank, stating customer risk profile, key financials, Credit History, Risk Factors with Analyst Note, Credit Decision & Explanation and Recommendations.</li> <li>Prepares a customer notification email with clear guidance and recommendations, personalized for the customer’s language and location when needed.</li> <li>Ensures all communications follow professional standards, with proper formatting and attention to compliance requirements.</li> </ul> By automating these steps, banks can accelerate credit risk decisoning system, improve communication, and ensure consistent, audit-ready documentation with detailed, explainable decisions and relevant insights for credit risk. </div> </div>

### 🛠️ Next Step
In the following section, we define the <code>credit_risk_report_generator</code> function, which will chain off of the <code>get_customer_details</code> function, in order to build an internal _and_ external credit risk report.

In [0]:
prompt = """
You are a Credit Risk Analyst Assistant for banks, helping to identify Customer Credit Risk and report the detailed, explainable decisions and recomendations.

### Tools Available:
1. **get_customer_details** – Retrieves customer details, financials transaction history, and risk indicators for a given **Customer ID**.

### Your Tasks:
1. Retrieve customer financial and behavior information.
2. Analyze risk score, payment patterns, and financial health.
3. If Prediction is 1 -> Customer Credit Risk score is high, If Prediction is 0 -> Customer Credit Risk score is low.
4. Generate a concise decision report that explains the outcome and provides your recommendation. 

Respond professionally and insightfully, structuring the report for easy consumption.
    - Ensure all details match the provided **Customer ID** before proceeding.

- **If the customer id is not present:**  
    - Briefly confirm that the customer is not present in the system .

- Do not provide your reasoning step; just give the response.
"""

In [0]:
spark.sql("DROP FUNCTION IF EXISTS credit_report_generator")

spark.sql(f"""
  CREATE OR REPLACE FUNCTION credit_report_generator(
    cust_id INT,
    education INT,
    marital_status INT,
    age INT,
    is_resident INT,
    months_current_address INT,
    months_employment INT,
    number_payment_delays_last12mo BIGINT,
    pct_increase_annual_number_of_delays_last_3_year BIGINT,
    overdraft_balance_amount DOUBLE,
    tot_rel_bal DOUBLE,
    tot_assets DOUBLE,
    income_annual INT,
    avg_balance DOUBLE,
    num_accs BIGINT,
    tot_txn_amt_12m DOUBLE,
    sent_txn_cnt_12m BIGINT,
    prediction DOUBLE
  )
  RETURNS STRING
  LANGUAGE SQL
  COMMENT "This function generates a credit risk report based on the customer details and customer financials information and credit score."
  RETURN (
    SELECT ai_query(
      'databricks-meta-llama-3-1-405b-instruct', 
      CONCAT(
        "{prompt}",
        'Customer details: ', 
        TO_JSON(NAMED_STRUCT(
              'cust_id', cust_id,
              'education', education,
              'marital_status', marital_status,
              'age', age,
              'is_resident', is_resident,
              'months_current_address', months_current_address,
              'months_employment', months_employment,
              'number_payment_delays_last12mo', number_payment_delays_last12mo,
              'pct_increase_annual_number_of_delays_last_3_year', pct_increase_annual_number_of_delays_last_3_year,
              'overdraft_balance_amount', overdraft_balance_amount,
              'tot_rel_bal', tot_rel_bal,
              'tot_assets', tot_assets,
              'income_annual', income_annual,
              'avg_balance', avg_balance,
              'num_accs', num_accs,
              'tot_txn_amt_12m', tot_txn_amt_12m,
              'sent_txn_cnt_12m', sent_txn_cnt_12m,
              'prediction', prediction
        ))
      )
    )
  )
""")

### Example:

Let's generate a report for a Credit Risk Analysis .

In [0]:
%sql
SELECT credit_report_generator(
    93407,0,0,45,1,30,97,0,0,158211467.36,7853.23,866, 86628,969,1,0,1,0
  ) as credit_report

## 🚀 Operation 3: Apply Our Functions in Batch to Customers with High Credit Risk  

<div style="display: flex; align-items: center; gap: 15px;">
  <img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" 
       style="width: 100px; border-radius: 10px;" />
  <div>
    Now that Liza has implemented the <code>get_customer_details</code> and <code>credit_report_generator</code> functions, the next step is to scale these operations by applying them in batch to a dataset of high risk customers.  
    This operation enables organizations to:
    <ul>
      <li>Process large volumes of historical or real-time transactions efficiently</li>
      <li>Generate detailed reports for flagged customers based on predefined criteria (e.g., unusual amounts or behavioral anomalies)</li>
      <li>Democratize access to enriched transaction data for auditors and credit risk analysts</li>
      <li>Identify patterns across multiple transactions for high  credit risk and risk analysis</li>
    </ul>
    By leveraging Databricks' scalable compute capabilities, this batch processing operation ensures that even large datasets can be analyzed quickly and effectively.
  </div>
</div>

### 🛠️ Next Step  
In the following section, we demonstrate how to apply the <code>get_customer_details</code> and <code>credit_report_generator</code> functions in batch mode to process suspicious transactions. This includes:
1. Loading a dataset of flagged customers from Unity Catalog.
2. Iteratively applying the functions to generate reports for each customer.
3. Storing the enriched reports back into Unity Catalog for downstream analysis.

<small>💡 Batch processing unlocks insights from historical financials transaction data, enabling proactive credit risk detection and improved operational efficiency through effective reporting system.</small>


In [0]:
%sql
CREATE OR REPLACE TABLE credit_report_generator_table AS
SELECT 
  cust_id AS cust_id,
  prediction,
  credit_report_generator(
    cust_id,
    education,
    marital_status,
    age,
    is_resident,
    months_current_address,
    months_employment,
    number_payment_delays_last12mo,
    pct_increase_annual_number_of_delays_last_3_year,
    overdraft_balance_amount,
    tot_rel_bal,
    tot_assets,
    income_annual,
    avg_balance,
    num_accs,
    tot_txn_amt_12m,
    sent_txn_cnt_12m,
    prediction
  ) AS credit_report
FROM (
  SELECT *
  FROM underbanked_prediction
  WHERE prediction = 1
  LIMIT 5
);

## Next Steps

Proceed to notebook **05.2-Agent-Creation-Guide** in order to package the above functions into an implementable AI Agent with Databricks!